In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import selenium.common.exceptions as sex

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import lxml

import time
from time import sleep
import random
import re

In [63]:
df = pd.read_csv('kyungki.csv')
df = df.drop(columns='Unnamed: 0', axis=1)
df = df.drop(columns='adress', axis=1)
df = df.drop(index=[0])
df.head()

,hospital,review_count,link,adr_clean
1,의료법인세창의료재단우성한의원,30,https://www.modoodoc.com/hospital/24116/%EC%9D...,경기
2,의료법인세창의료재단우성한의원,30,https://www.modoodoc.com/hospital/24116/%EC%9D...,경기
3,누베베한의원,20,https://www.modoodoc.com/hospital/22565/%EB%88...,경기
4,의료법인자인의료재단자인메디병원,19,https://www.modoodoc.com/hospital/17522/%EC%9D...,경기
5,모커리한방병원,19,https://www.modoodoc.com/hospital/21871/%EB%AA...,경기


In [64]:
df.review_count.value_counts()

1     713
2     430
3     200
4     110
5      70
6      51
7      33
8      15
12      9
9       9
14      6
10      5
16      5
11      4
13      3
15      3
19      2
30      2
20      1
Name: review_count, dtype: int64

In [65]:
df = df[df['review_count']>=10]
print(df.shape)

(40, 4)


In [66]:
df

,hospital,review_count,link,adr_clean
1,의료법인세창의료재단우성한의원,30,https://www.modoodoc.com/hospital/24116/%EC%9D...,경기
2,의료법인세창의료재단우성한의원,30,https://www.modoodoc.com/hospital/24116/%EC%9D...,경기
3,누베베한의원,20,https://www.modoodoc.com/hospital/22565/%EB%88...,경기
4,의료법인자인의료재단자인메디병원,19,https://www.modoodoc.com/hospital/17522/%EC%9D...,경기
5,모커리한방병원,19,https://www.modoodoc.com/hospital/21871/%EB%AA...,경기
6,경희약손한의원,16,https://www.modoodoc.com/hospital/22178/%EA%B2...,경기
7,미올한의원,16,https://www.modoodoc.com/hospital/23014/%EB%AF...,경기
8,경희365늘한의원,16,https://www.modoodoc.com/hospital/22009/%EA%B2...,경기
9,누베베한의원,16,https://www.modoodoc.com/hospital/22566/%EB%88...,경기
10,거북이한의원,16,https://www.modoodoc.com/hospital/21979/%EA%B1...,경기


In [69]:
df_v2 = df[2:35]
df_v2

,hospital,review_count,link,adr_clean
3,누베베한의원,20,https://www.modoodoc.com/hospital/22565/%EB%88...,경기
4,의료법인자인의료재단자인메디병원,19,https://www.modoodoc.com/hospital/17522/%EC%9D...,경기
5,모커리한방병원,19,https://www.modoodoc.com/hospital/21871/%EB%AA...,경기
6,경희약손한의원,16,https://www.modoodoc.com/hospital/22178/%EA%B2...,경기
7,미올한의원,16,https://www.modoodoc.com/hospital/23014/%EB%AF...,경기
8,경희365늘한의원,16,https://www.modoodoc.com/hospital/22009/%EA%B2...,경기
9,누베베한의원,16,https://www.modoodoc.com/hospital/22566/%EB%88...,경기
10,거북이한의원,16,https://www.modoodoc.com/hospital/21979/%EA%B1...,경기
11,수인한의원,15,https://www.modoodoc.com/hospital/23612/%EC%88...,경기
12,차이플러스한의원,15,https://www.modoodoc.com/hospital/24467/%EC%B0...,경기


In [70]:
root_url = df_v2.link.tolist()
print(len(root_url))

33


In [71]:
urls_list=[]

for j in root_url:
    root_url = j
    
    for i in range(1,3):
#         print(root_url)
        urls = root_url+('?page='+str(i))
        urls_list.append(urls)
        print(urls)
        
print(len(urls_list))

https://www.modoodoc.com/hospital/22565/%EB%88%84%EB%B2%A0%EB%B2%A0%ED%95%9C%EC%9D%98%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EC%95%BC%ED%83%91%EB%8F%99/?page=1
https://www.modoodoc.com/hospital/22565/%EB%88%84%EB%B2%A0%EB%B2%A0%ED%95%9C%EC%9D%98%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EC%95%BC%ED%83%91%EB%8F%99/?page=2
https://www.modoodoc.com/hospital/17522/%EC%9D%98%EB%A3%8C%EB%B2%95%EC%9D%B8%EC%9E%90%EC%9D%B8%EC%9D%98%EB%A3%8C%EC%9E%AC%EB%8B%A8%EC%9E%90%EC%9D%B8%EB%A9%94%EB%94%94%EB%B3%91%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%ED%96%89%EC%8B%A0%EB%8F%99/?page=1
https://www.modoodoc.com/hospital/17522/%EC%9D%98%EB%A3%8C%EB%B2%95%EC%9D%B8%EC%9E%90%EC%9D%B8%EC%9D%98%EB%A3%8C%EC%9E%AC%EB%8B%A8%EC%9E%90%EC%9D%B8%EB%A9%94%EB%94%94%EB%B3%91%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%ED%96%89%EC%8B%A0%EB%8F%99/?page=2
https://www.modoodoc.com/hospital/21871/%EB%AA%A8%EC%BB%A4%EB%A6%AC%ED%95%9C%EB%B0%A9%EB%B3%91%EC%9B%90-%EC%88%98%EC%9B%90%EC%A0%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EA%B5%AC%EC%B2%9C%EB%8F

In [42]:
path = '/Users/monica_air/chromedriver'
driver = webdriver.Chrome(path)
driver.implicitly_wait(5)

start_url = 'https://www.modoodoc.com/hospitals/?search_query=%ED%95%9C%EC%9D%98%EC%9B%90'
driver.get(start_url)

/var/folders/kx/wntkz3hj61bd2_ss_s_4d1km0000gn/T/ipykernel_7106/2044224742.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [45]:
urls

'https://www.modoodoc.com/hospital/24690/%ED%8F%89%EC%B4%8C%EB%B6%80%EB%B6%80%ED%95%9C%EC%9D%98%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EA%B4%80%EC%96%91%EB%8F%99/?page=2'

In [73]:
np.random.rand(5)

array([0.50593265, 0.26676414, 0.69309841, 0.13643635, 0.27762893])

In [86]:
reviews = []
stars= []
hoss = []
z=0
j=0
for i in urls_list[1:3]:
#     j=0
#     print(i)
    driver.get(i)
    driver.implicitly_wait(5)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    
    hos = soup.find_all('h1', class_="my-1 hospital-doctor-name-box d-flex align-items-center")
    for i in hos:
        hos = i.text
        hos = hos.strip()
#         print('\n>>>>>>>>>>>>>>       {}      <<<<<<<<<<<<<<'.format(hos))
        hoss.append(hos)
    
    
    review = soup.find_all('div', class_="toast-message content-box text-left class49 mb-4 pb-2")

   

    for i in review:
        text = i.text
        #     print(text)
        text = text.replace('\n', '')
        text = text.replace('                        ', '')
        text = text.replace('                    ', '')
        z+=1
#         print(str(z)+' //'+hos+'// '+text)
#         print("reviews extracted")
       
        reviews.append(str(z)+' //'+hos+'// '+text)
        

    star = soup.find_all('span', class_ = "h-100 align-bottom")

    
    for i in star:
        i = i.text
        i=i.strip()
        stars.append(i)
#         print(i)
    print("Stared")
        
    j+=1
    
    print('------------     남은 페이지수 {}     ----------------'.format(len(root_url)-j))
    
    x = random.randint(0,2)
    sleep(x)
            
    print('ㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇ sleep for {} ㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇ'.format(x))

10
9
7.3
8.3
9
10
8
9
8
9
Stared
------------     남은 페이지수 159     ----------------
ㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇ sleep for 2 ㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇ
6
9
7.8
8
8
10
10
4.3
9.3
7
Stared
------------     남은 페이지수 158     ----------------
ㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇ sleep for 0 ㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇ


In [87]:
print(len(reviews))
print(len(stars))
print(len(hoss))

20
20
2


In [88]:
reviews

['1 //누베베한의원// 다이어트를 위해서 방문을 했었답니다. 건강에 위협이 될정도로 살이 졌는데 빠지지는 않다보니 병원에 방문해서 한약을 지어먹으려고 갔었답니다. 서울에 있다보니 가기가 너무 힘들지만 검사를 받은 후부터는 전화상으로도 상담이 가능하고, 약도 추가로 하고싶으면 가능해서 너무 좋았었답니다.약을 먹기전에 주의사항이나 복용법을 자세히 알려주셔서 너무 좋았구요. 약이 정말 효과가 좋아서 몸무게가 10키로이상 빠졌었답니다!',
 '2 //누베베한의원// 다이어트기 힘들어서 주변에서 추천을 많이 한 누베베 방문 했어요~ 정말 친절하고 사전 검사 꼼꼼하게 해주셔서 잘 받을 수 있었어요. 아직 한달 정도 먹었는데 3키로 빠지고 두달 남았으니 더더 큰 효과 기대 합니다. 다른 곳은 두통 손떨림 등 있다던데 그런 증상 전혀 없이 편하게 먹고 있어요~부작용은 딱히 없고 처음엔 밤에 잠이 좀 안 오긴 하나 시간 지나니 괜찮네요~',
 '3 //누베베한의원// 다이어트한약으로 유명한 병원입니다. 저는 생리통 완화 한약을 처방받으러 방문하였습니다. 사전 문진사항이 꽤 많고 인바디 측정도 합니다. 진맥한 후에 여러가지를 물어보고 약 처방해주시구요~ 약은 환으로 되어 있어서 복용은 편합니다. 3달동안 먹는거였는데 한달에 한번 가서 차도가 얼마나 있나 또 문진 작성하고 약 처방 받습니다. 개인마다 차이는 있겠지만 저는 생리통이 심한 편이고 이전에 다른 한의원에서도 생리통 한약 먹었는데 크게 효과보지 못했고 누베베한의원의 약을 먹고도 크게 효과는 보지 못했습니다. 직원들이나 의사쌤은 매우 친절하십니다',
 '4 //누베베한의원// 누베베 분당점 김기현 한의사님에게 상담 받았습니다. 예약을 하고 가서 대기시간은 짧지 않았습니다. 인바디와 스트레스 검사 등을 하고 주의사항을 영상으로 짧게 시청하였습니다. 검사결과를 보며 상담을 진행했는데 목표체중을 정해주시고 지켜야할 사항들과 식단 등을 설명해주셨습니다. 약은 일주일 뒤에 집으로 배송되는 형태인데.. (재진자에 한해서?) 요즘엔

In [89]:
stars

['10',
 '9',
 '7.3',
 '8.3',
 '9',
 '10',
 '8',
 '9',
 '8',
 '9',
 '6',
 '9',
 '7.8',
 '8',
 '8',
 '10',
 '10',
 '4.3',
 '9.3',
 '7']

In [ ]:
review = soup.find_all('div', class_="toast-message content-box text-left class49 mb-4 pb-2")

reviews = []

for i in review:
    text = i.text
#     print(text)
    text = text.replace('\n', '')
    text = text.replace('                        ', '')
    text = text.replace('                    ', '')
#     print(text)
    reviews.append(text)
    
star = soup.find_all('span', class_ = "h-100 align-bottom")

stats= []
for i in star:
    i = i.text
    i=i.strip
    print(i)
    
print(reviews)

In [23]:
driver.get(root_url[3])
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')



hos = soup.find_all('h1', class_="my-1 hospital-doctor-name-box d-flex align-items-center")
for i in hos:
    i = i.text
    i = i.strip()
    print(i)

의료법인자인의료재단자인메디병원


In [85]:
driver.get(urls_list[3])
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

star = soup.find_all('span', class_ = "h-100 align-bottom")

stats= []
for i in star:
    i = i.text
    
    i=i.strip()
#     print(i)
    print(i)
    

10
10
9
9
10
7.8
10
8
10


In [7]:
adr = soup.find_all('div', class_="color49 mt-3")
for i in adr:
    i = i.text
    i = i.strip()
    print(i)

In [15]:
for i in range(1,4):
    print(i)

1
2
3


In [16]:
reviews=[]
stars= []
treatments=[]
hospitals = []
adr = []

for i in root_url:
    base_url = i
    # print(base_url)

    for i in range(1,4):
        i=0
        urls = base_url+('?page='+str(i))
        # print(base_url)
        try :
            urls = base_url+('?page='+str(i))
            print(urls)
        

            driver.get(urls)
            driver.implicitly_wait(5)
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')


            review = soup.find_all('div', class_="toast-message content-box text-left class49 mb-4 pb-2")

            

            for i in review:
                text = i.text
            #     print(text)
                text = text.replace('\n', '')
                text = text.replace('                        ', '')
                text = text.replace('                    ', '')
                # print(text)
                reviews.append(text)

            star = soup.find_all('span', class_ = "h-100 align-bottom")

            
            for i in star:
                i = i.text
                i=i.strip()
                stars.append(i)
                
                
            treatment = soup.find_all('div', class_="text-left mt-2 title-box")
            
            
            for i in treatment:
                treat = i.text
                treat = treat.replace(' ','')
                treat = treat.replace('\n','')
                treat = treat.split(':')[1]
                treatments.append(treat)
                
#             hos = soup.find_all('h1', class_="my-1 hospital-doctor-name-box d-flex align-items-center")
#             for i in hos:
#                 i = i.text
#                 i = i.strip()
#                 hospitals.append(i)
                
#             adr = soup.find_all('div', class_="color49 mt-3")
#             for i in adr:
#                 i = i.text
#                 i = i.strip()
#                 adr.append(i)
            
            print('10 review extracted')
            
#             x = random.randint(0,2)
#             sleep(x)
            
#             print('sleep for', x)

            i+=1
            print('--------      {}    ----------'.format(i))
        
        
        
        except :
            print('리뷰없음')
            pass

https://www.modoodoc.com/hospital/24116/%EC%9D%98%EB%A3%8C%EB%B2%95%EC%9D%B8%EC%84%B8%EC%B0%BD%EC%9D%98%EB%A3%8C%EC%9E%AC%EB%8B%A8%EC%9A%B0%EC%84%B1%ED%95%9C%EC%9D%98%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EA%B3%A0%EC%9E%94%EB%8F%99/?page=0
10 review extracted
리뷰없음
https://www.modoodoc.com/hospital/24116/%EC%9D%98%EB%A3%8C%EB%B2%95%EC%9D%B8%EC%84%B8%EC%B0%BD%EC%9D%98%EB%A3%8C%EC%9E%AC%EB%8B%A8%EC%9A%B0%EC%84%B1%ED%95%9C%EC%9D%98%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EA%B3%A0%EC%9E%94%EB%8F%99/?page=0
10 review extracted
리뷰없음
https://www.modoodoc.com/hospital/24116/%EC%9D%98%EB%A3%8C%EB%B2%95%EC%9D%B8%EC%84%B8%EC%B0%BD%EC%9D%98%EB%A3%8C%EC%9E%AC%EB%8B%A8%EC%9A%B0%EC%84%B1%ED%95%9C%EC%9D%98%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EA%B3%A0%EC%9E%94%EB%8F%99/?page=0
10 review extracted
리뷰없음
https://www.modoodoc.com/hospital/24116/%EC%9D%98%EB%A3%8C%EB%B2%95%EC%9D%B8%EC%84%B8%EC%B0%BD%EC%9D%98%EB%A3%8C%EC%9E%AC%EB%8B%A8%EC%9A%B0%EC%84%B1%ED%95%9C%EC%9D%98%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%

리뷰없음
https://www.modoodoc.com/hospital/21850/%EB%8F%84%EC%86%94%ED%95%9C%EB%B0%A9%EB%B3%91%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EB%B9%84%EC%A0%84%EB%8F%99/?page=0
리뷰없음
https://www.modoodoc.com/hospital/24263/%EC%9E%90%EC%9A%B0%EB%8B%B9%EB%B6%80%EB%B6%80%ED%95%9C%EC%9D%98%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EC%9A%A9%EC%9D%B4%EB%8F%99/?page=0
리뷰없음
https://www.modoodoc.com/hospital/24263/%EC%9E%90%EC%9A%B0%EB%8B%B9%EB%B6%80%EB%B6%80%ED%95%9C%EC%9D%98%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EC%9A%A9%EC%9D%B4%EB%8F%99/?page=0
리뷰없음
https://www.modoodoc.com/hospital/24263/%EC%9E%90%EC%9A%B0%EB%8B%B9%EB%B6%80%EB%B6%80%ED%95%9C%EC%9D%98%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EC%9A%A9%EC%9D%B4%EB%8F%99/?page=0
리뷰없음
https://www.modoodoc.com/hospital/23467/%EC%84%B1%EB%82%A8%ED%95%9C%EC%9D%98%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EC%8B%A0%ED%9D%A5%EB%8F%99/?page=0
리뷰없음
https://www.modoodoc.com/hospital/23467/%EC%84%B1%EB%82%A8%ED%95%9C%EC%9D%98%EC%9B%90-%EA%B2%BD%EA%B8%B0%EB%8F%84-%EC%8B%A0%ED%9D%

In [13]:

print(len(reviews))
print(len(stars))
print(len(treatments))
print(len(hospitals))

4898
4898
4358
873


In [14]:
# kyungki_review = pd.DataFrame({'reviews':reviews, 'star': stars, 'treatments':treatments})
# kyungki_review.to_csv('kyungki_reviews.csv')
# kyungki_review

In [15]:
# kyungki_review = pd.DataFrame({'reviews':reviews, 'star': stars, 'treatments':treatments})
# kyungki_review.to_csv('kyungki_reviews.csv')

In [16]:
import pickle
with open ('reviews.pkl','wb') as f:
    pickle.dump(reviews, f)

with open ('stars.pkl','wb') as f:
    pickle.dump(stars, f)

with open ('treatments.pkl','wb') as f:
    pickle.dump(treatments, f)

In [17]:
with open('reviews.pkl', 'rb') as f:
    reviews_check = pickle.load(f)

with open('stars.pkl', 'rb') as f:
    stars_check = pickle.load(f)

with open('treatments.pkl', 'rb') as f:
    treatments_check = pickle.load(f)


print(reviews_check)
print(stars_check)
print(treatments_check)

['생리를 하지 않아 방문햇습니다 다낭성난소증후군이 의심된다며 체중감소를 해보자고 하셧어요 피임약으로 약물치료 해야한다고 알려주셧고일단 생리유도제 맞고 생리 후 다시 오라고 하셨어요 ~', '자궁경부암검사 받으러 갔었어요.친절하시고 불편하지않게 진료 후딱 끝내주셨어요.간호사선샹님, 상담선생님들도 친절해요.건강검진도 무료대상자라서 아침밥 안먹었더니깐 건강검진도 권해주셔서 같이 했어요', '우성산부인과 검진받으러 갔어요. 건물은 노후되어 있었지만 꼼꼼히 봐주셨어요선생님과 직원은 친절했습니다', '집앞이라서 경부암 검사 및 여성질환관련해서진료보러 종종 가는거 같아요출산하는 병원이라 대기 시간이 길때도 있지만직장 다니면서 진료보려면 진료시간도 걸리는데생각보다 늦게까지 진료하시고 주말진료도 있어 좋아요', '국가검진으로 인해 다녀왔습니다 근처에서 잘 이용하던 곳이였습니다. 믿고있는 만큼 정기적으로 잘 다니고 있습니다 주차공간이 많지 않아서 불편함이 있지만 대체적으로 잘 관리되고 한방의료여서 더욱신뢰갑니다', '대기시간은 있지만언제나 믿고 신뢰할수있는 병원입니다친절한 상담에서부터 마음을 놓을 수 있구요진료도 친절하게 잘 해주셔서 항상 부담없이 다니고 있어요산과와 부인과 모두 추천입니다', '독감 백신 맞으러 갔어요. 친구 아기 예방 접종 맞으러 간 김에 저도 맞았어요. 아기들이 한창 예방 접종 맞을 시기여서 꽤나 기다렸어요. 그래도 그때 백신이 없어서 못 맞을때였어서 맞고 가자 해서 맞았어요.', '제가 오후늦게 방문해서 당직쌤이 봐주셨어요이전에 산전검사햇었고풍진항체가 없어서 주사를 맞으러 갔구요선생님께서 관리사항이나 주의사항에 대해 자세하게설명해주셨어요주사 놔주시는 분도 친절했어요', '남자선생님보다는 여자선생님을 선호하는편인데 수술을 잘하신다고 접수할때 추천받았습니다 예약하고가면 대기시간이 길지않고,선생님은 물론 간호사선생님들도 바쁘신데도 친절하시고 설명도 잘해주셔서 좋았습니다', '자궁경부암 건강검진으로 우성병원 방문했어요. 일부로 여자 의사선생님으로 진료 받고 싶

In [18]:
with open ('hospitals_check.pkl','wb') as f:
    pickle.dump(hospitals, f)
with open('hospitals_check.pkl', 'rb') as f:
    hospitals_check = pickle.load(f)

In [19]:
print(hospitals_check)

['의료법인세창의료재단우성한의원', '의료법인세창의료재단우성한의원', '의료법인세창의료재단우성한의원', '의료법인세창의료재단우성한의원', '의료법인세창의료재단우성한의원', '의료법인세창의료재단우성한의원', '누베베한의원', '누베베한의원', '누베베한의원', '의료법인자인의료재단자인메디병원', '의료법인자인의료재단자인메디병원', '의료법인자인의료재단자인메디병원', '모커리한방병원 수원점', '모커리한방병원 수원점', '모커리한방병원 수원점', '경희약손한의원', '경희약손한의원', '경희약손한의원', '미올한의원 의정부점', '미올한의원 의정부점', '미올한의원 의정부점', '경희365늘한의원', '경희365늘한의원', '경희365늘한의원', '누베베한의원', '누베베한의원', '누베베한의원', '거북이한의원', '거북이한의원', '거북이한의원', '수인한의원', '수인한의원', '수인한의원', '차이플러스한의원', '차이플러스한의원', '차이플러스한의원', '수여성병원', '수여성병원', '수여성병원', '수원자생한방병원', '수원자생한방병원', '수원자생한방병원', '365드림의원', '365드림의원', '365드림의원', '마디마디한의원', '마디마디한의원', '마디마디한의원', '사랑인한의원', '사랑인한의원', '사랑인한의원', '동천한의원', '동천한의원', '동천한의원', '일산자생한방병원', '일산자생한방병원', '일산자생한방병원', '모아한의원 부천점', '모아한의원 부천점', '모아한의원 부천점', '도솔한방병원', '도솔한방병원', '도솔한방병원', '자우당부부한의원', '자우당부부한의원', '자우당부부한의원', '성남한의원', '성남한의원', '성남한의원', '바른생한의원', '바른생한의원', '바른생한의원', '한마음한의원', '한마음한의원', '한마음한의원', '광명 맥심한의원', '광명 맥심한의원', '광명 맥심한의원', '안산자생한방병원', '안산자생한방병원', '안산자생한방병원', '원진녹색병원', '원진녹색병원', '원진녹색병원